In [1]:
import numpy as np
import cv2
from collections import deque

In [55]:
cap=cv2.VideoCapture('volleyball_match.mp4')
cap.isOpened()

True

In [41]:
yellow_players_list=deque(maxlen=140)
red_players_list=deque(maxlen=140)
y,r=0,0

In [4]:
frame_width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps=int(cap.get(cv2.CAP_PROP_FPS))
fps,frame_height,frame_width

(28, 720, 1280)

In [5]:
saver=cv2.VideoWriter('final_output.mp4',cv2.VideoWriter_fourcc(*'mp4v'),fps,(frame_width,frame_height))
bg_sub=cv2.createBackgroundSubtractorKNN(history=60,dist2Threshold=10000,detectShadows=False)

In [6]:
def score(hull):
    area = cv2.contourArea(hull)
    perimeter = cv2.arcLength(hull, True)
    x, y, w, h = cv2.boundingRect(hull)
    aspect_ratio = float(w) / h
    
    if perimeter > 0 and 150 < area < 220:
        circularity = (4 * np.pi * area) / (perimeter ** 2)
        (encircle_x, encircle_y), radius = cv2.minEnclosingCircle(hull)
        fit_ratio = area / (np.pi * radius ** 2) 
    
        score = (circularity * 0.2) + (fit_ratio * 0.3) + ((0.9 <= aspect_ratio <= 1.1) * 0.1) 
        return -1*score
    return 0

In [63]:
while cap.isOpened():
    _,frame=cap.read()

    if not _:
        break

    loss=np.inf
    
    hsv_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    lower_bound = np.array([8, 100, 100])
    upper_bound = np.array([40, 255, 255]) 
    
    hsv_mask=cv2.inRange(hsv_frame,lower_bound,upper_bound)
    hsv_mask=cv2.GaussianBlur(hsv_mask,(3,3),0)
    
    mask=hsv_mask
    
    contours,_=cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        hull=cv2.convexHull(contour)
        loss_now=score(hull)
        if loss_now<loss:
            best_hull=hull
            loss=loss_now
    (x,y),rad=cv2.minEnclosingCircle(best_hull)
    cv2.circle(frame,(int(x),int(y)),int(rad),(0,255,0),3)

    yellow_lower = np.array([5, 0, 100])
    yellow_upper = np.array([67, 255, 255])
    red_lower = np.array([175, 137, 100])
    red_upper = np.array([178, 255, 255])
    white_lower=np.array([0,0,0])
    white_upper=np.array([0,0,0])

    yellow_mask = cv2.inRange(hsv_frame, yellow_lower, yellow_upper)
    red_mask = cv2.inRange(hsv_frame, red_lower, red_upper)
    white_mask = cv2.inRange(hsv_frame, white_lower, white_upper)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    
    yellow_mask = cv2.morphologyEx(yellow_mask, cv2.MORPH_OPEN, kernel)
    red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, kernel)
    white_mask = cv2.morphologyEx(white_mask, cv2.MORPH_OPEN, kernel)

    yellow_contours, _ = cv2.findContours(yellow_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    red_contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    white_contours,_=cv2.findContours(white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    

    yellow_players = 0
    red_players = 0  
    for contour in yellow_contours:
        area = cv2.contourArea(contour)
        if area > 500: 
            yellow_players += 1

    for contour in red_contours:
        area = cv2.contourArea(contour)
        if area > 500: 
            red_players += 1
    for contour in white_contours:
        area = cv2.contourArea(contour)
        if area > 500: 
            red_players += 1
    
    yellow_players_list.append(yellow_players)
    red_players_list.append(red_players)

    avg_yellow_players = sum(yellow_players_list) / len(yellow_players_list) if yellow_players_list!=0 else 0
    avg_red_players = sum(red_players_list) / len(red_players_list) if red_players_list!=0 else 0


    cv2.putText(frame, f"Number of yellow players: {yellow_players}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2) # Yellow color
    cv2.putText(frame, f"Number of red players: {red_players}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)   # Red color
    cv2.putText(frame, f"Number of yellow players (averaged over 5s: {int(avg_yellow_players)}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2) # Yellow color
    cv2.putText(frame, f"Number of red players (averaged over 5s): {int(avg_red_players)}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2) # Red color
    
    cv2.imshow('hsv_mask',hsv_mask)
    cv2.imshow('mask',mask)
    # cv2.drawContours(frame,contours,-1,(0,255,0),3)
    cv2.imshow('frame',frame) 
    if cv2.waitKey(1)&0xFF==ord('q')|ord('Q'):
    # cv2.waitKey(0)
        cv2.destroyAllWindows()
        break
    # cv2.waitKey(200)
cv2.destroyAllWindows()

<h1>Determining the best bound values</h1>

In [23]:
_,frame=cap.read()

def nothing(X):
    pass

cv2.namedWindow('adjusters')
cv2.createTrackbar('LH','adjusters',0,255,nothing)
cv2.createTrackbar('LS','adjusters',0,255,nothing)
cv2.createTrackbar('LV','adjusters',0,255,nothing)
cv2.createTrackbar('HH','adjusters',0,255,nothing)
cv2.createTrackbar('HS','adjusters',0,255,nothing)
cv2.createTrackbar('HV','adjusters',0,255,nothing)

while True:
    LH=cv2.getTrackbarPos('LH','adjusters')
    LV=cv2.getTrackbarPos('LV','adjusters')
    LS=cv2.getTrackbarPos('LS','adjusters')
    UH=cv2.getTrackbarPos('HH','adjusters')
    US=cv2.getTrackbarPos('HS','adjusters')
    UV=cv2.getTrackbarPos('HV','adjusters')

    
    lower_bound = np.array([LH, LS, LV])
    upper_bound = np.array([UH, US, UV]) 

    hsv_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

    hsv_mask=cv2.inRange(hsv_frame,lower_bound,upper_bound)
    hsv_mask=cv2.GaussianBlur(hsv_mask,(3,3),0)
    
    cv2.imshow('hsv_mask',hsv_mask)
    cv2.imshow('frame',frame) 
    if cv2.waitKey(1)&0xFF==ord('q'):
    # cv2.waitKey(0)
        cv2.destroyAllWindows()
        break

Values used above were determined to be the best